# Load Data

In [ ]:
import tensorflow as tf

# Load the Fashion MNIST dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the data
x_train, x_test = x_train / 255.0, x_test / 255.0

## Plot Data

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))

for i in range(8):
    plt.subplot(2, 4, i + 1)
    plt.imshow(x_train[i], cmap='gray')
    plt.title(f'Label: {y_train[i]}')
    plt.axis('off')

plt.show()

In [ ]:
# Check the shape of the data
# Each image is 28 x 28 pixels
print(x_train[1].shape)

# Random Forest

## Data Prep

In [ ]:
# Many machine learning models expect data to be in a 2D table, where each row is a sample and each column is a feature.
# We can flatten the 28x28 images into a single row of 784 pixels.

# So, first we need to prepare/alter the images so that a random forest can use them.
# The random forest will use the pixel values as features, so give it a dataset of flat vectors like
# the one we used for the fashion_mnist example.

'Your code here'

In [ ]:
# There are 10 classes in the MNIST dataset, just like the fashion_MNIST.
# You need to select 3 classes to train your model on. You get to pick the classes you want to use, but 
# Please choose classes that are similar to each other.
# For example, you could choose 3, 5, and 8. Or 4, 7, 9. Or other combinations that you deam appropriate.
# Please don't choose digits that are easy to tell apart, like 0, 1, and 5. Or 1, 2, and 3.

# You can totally copy code over from the fmnist_examples notebook and modify it.
# You can use chatGPT, copilot, google, or other AI or online resources.
# Use each other, the goal is to complete the objective and maybe learn something new, not to struggle to make up code on your own.
# We have prior notebooks, chatGPT, the internet, and each other for the rapids projects, so use them here if they will help!

# Extract the 3 classes from the training data

'Your code here'

# Extract the 3 classes from the test data

'Your code here'

# Maybe check the shape of the data is correct

'Your code here if you want'


## Simple Model Fit

In [ ]:
# Fit a simple random forest. Make sure it has at least 100 trees, Probably not more than 1000 trees.

'Your code here'

# Generate predictions on the test set and calculate the overall accuracy of the model on the test set.

'Your code here'


# Tune Model Fit

In [ ]:
# Now try to tuning the model. 
# This part can take a while, maybe several minutes.

'Your code here'

# Generate predictions on the test set and calculate the overall accuracy of the model on the test set.

'Your code here'

# Does tuning the model improve the accuracy? How much?
# Was the tuning accuracy worth the extra time?

In [ ]:
# Compute and plot a confusion matrix.

'Your code here'

# Where are misclassifications occurring?

In [ ]:
# Plot some of the misclassifications.

'Your code here'

# Can you see why the model made the misclassifications it did?